In [2]:
!pip install opencv-python -i https://pypi.douban.com/simple
!pip install pillow -i https://pypi.douban.com/simple
!pip install --upgrade tqdm
!pip install efficientnet

import pickle
import glob
from multiprocessing import Pool
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
import concurrent.futures
from PIL import Image
from skimage import io
import tensorflow as tf

import cv2
import pandas as pd
import os
from tqdm import tqdm
import time
import json
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D,GlobalAveragePooling2D,Convolution2D,BatchNormalization, Activation,MaxPooling2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
from tensorflow import keras
import efficientnet.tfkeras as efn
from keras.applications.resnet50 import ResNet50
from keras.applications import InceptionV3,MobileNetV2,InceptionResNetV2
from keras.applications.resnet import ResNet101
import tensorflow.keras.backend as K
tf.compat.v1.disable_eager_execution()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
config = tf.compat.v1.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config = config)
print(sess)

Looking in indexes: https://pypi.douban.com/simple
Looking in indexes: https://pypi.douban.com/simple
Requirement already up-to-date: tqdm in /opt/conda/lib/python3.8/site-packages (4.50.2)
Num GPUs Available:  1
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:07.0, compute capability: 7.5



In [14]:
def Model():
    covn_base = InceptionV3(input_shape=(224,224,3),include_top=False,
                            pooling='max',
                            weights='imagenet'
                            )
#     covn_base.trainable = True
    model = keras.Sequential()
    model.add(covn_base)
    model.add(keras.layers.Dense(3,activation='softmax'))
    adam=tf.keras.optimizers.Adam(lr=3e-4,
            # beta_1=0.9, 
            # beta_2=0.999,
            # epsilon=None,
            # decay=0.0, 
            # amsgrad=False
            )
    model.compile(optimizer=adam,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'],
              )
    return model

In [3]:
from tensorflow.keras import *
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
def scheduler(epoch):
    # 每隔100个epoch，学习率减小为原来的1/10
    if epoch % 100 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    return K.get_value(model.optimizer.lr)
 
# reduce_lr = LearningRateScheduler(scheduler)

In [17]:
import numpy as np

Batch_s=64
# # 训练集0.8
# train_data = tf.keras.preprocessing.image_dataset_from_directory(
#     '/home/kesci/work/picture_data',
#     validation_split = 0.2,
#     subset='training',
#     seed=3090,
#     image_size=(224,224),
#     batch_size=Batch_s
# )
# # 验证集0.2
# val_data = tf.keras.preprocessing.image_dataset_from_directory(
#     '/home/kesci/work/picture_data',
#     validation_split = 0.2,
#     subset='validation',
#     seed=3090,
#     image_size=(224,224),
#     batch_size=Batch_s
# )
def random_crop_image(image):
      height, width = image.shape[:2]
    #   random_array = np.random.uniform(0.6,0.7)
      random_array=0.7
      y = int(random_array*height)
      image_crop = image[0:y,0:width,0:3]
      image_crop = cv2.resize(image_crop,(224, 224))
      return image_crop
    
train_datagen = ImageDataGenerator(rescale=1./255,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # brightness_range=[0.1,0.3],
    # preprocessing_function=random_crop_image,
    validation_split=0.2)


test_datagen = ImageDataGenerator(rescale=1./255,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # brightness_range=[0.1,0.3],
    # preprocessing_function=random_crop_image,
    validation_split=0.2)

# 训练集0.8
train_data = train_datagen.flow_from_directory(
    '/home/kesci/work/picture_data',
    subset='training',
    class_mode='sparse',
    # seed=3090,
    target_size=(224,224),
    batch_size=Batch_s)

# 验证集0.2
val_data = test_datagen.flow_from_directory(
    '/home/kesci/work/picture_data',
    class_mode='sparse',
    subset='validation',
    # seed=3090,
    target_size=(224,224),
    batch_size=Batch_s)

EarlyStop=EarlyStopping(monitor='val_accuracy',
                        patience=8,verbose=1, mode='max')
#减小学习率
Reduce=ReduceLROnPlateau(monitor='val_accuracy',
                         factor=0.2,
                         patience=2,
                         verbose=1,
                         mode='max',
                        #  min_delta=0.0001,
                         cooldown=0,
                         min_lr=0
                         )
                                                
checkpoint = ModelCheckpoint(f'/home/kesci/work/cnn_model/inceptionnet.h5',
                            monitor='val_accuracy',
                            verbose=0,
                            mode='max',
                            save_best_only=True)
# NUM_TRAIN=141668
# NUM_TEST=35417
model=Model()

model.fit(
    train_data,
    steps_per_epoch=train_data.samples//Batch_s,
    epochs=10,
    callbacks=[Reduce,checkpoint],
    validation_data=val_data,
    validation_steps=val_data.samples//Batch_s
    )


Found 141669 images belonging to 3 classes.
Found 35416 images belonging to 3 classes.
Epoch 1/10
2213/2213 [==============================] - 2119s 957ms/step - batch: 1106.0000 - size: 63.9878 - loss: 0.5817 - accuracy: 0.7663 - val_loss: 0.5931 - val_accuracy: 0.7751
Epoch 2/10
2213/2213 [==============================] - 1416s 640ms/step - batch: 1106.0000 - size: 63.9878 - loss: 0.5138 - accuracy: 0.7931 - val_loss: 0.5528 - val_accuracy: 0.7651
Epoch 3/10
2213/2213 [==============================] - ETA: 0s - batch: 1106.0000 - size: 63.9878 - loss: 0.4832 - accuracy: 0.8037
Epoch 00003: ReduceLROnPlateau reducing learning rate to 6.000000284984708e-05.
2213/2213 [==============================] - 1437s 649ms/step - batch: 1106.0000 - size: 63.9878 - loss: 0.4832 - accuracy: 0.8037 - val_loss: 1.0446 - val_accuracy: 0.6461
Epoch 4/10
2213/2213 [==============================] - 1417s 640ms/step - batch: 1106.0000 - size: 63.9878 - loss: 0.3849 - accuracy: 0.8423 - val_loss: 0.436

KeyboardInterrupt: 

In [3]:
# 测试集预测
test_datagen = ImageDataGenerator(
    rescale=1./255,
    )
test_generator = test_datagen.flow_from_directory(
    '/home/kesci/work/test_data', 
    target_size=(224, 224),
    batch_size=64,
    class_mode='sparse', 
    shuffle=False,)
test_generator.reset()


Found 72778 images belonging to 1 classes.


In [4]:
model = tf.keras.models.load_model('/home/kesci/work/cnn_model/inceptionnet.h5')

In [5]:
pred = model.predict(test_generator, verbose=1)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


In [26]:
pred

array([[0.5264642 , 0.07702505, 0.39651078],
       [0.00889829, 0.04002218, 0.95107955],
       [0.17420694, 0.08438879, 0.74140424],
       ...,
       [0.37153983, 0.37002924, 0.25843093],
       [0.65803283, 0.31800178, 0.02396537],
       [0.06305029, 0.9315972 , 0.00535253]], dtype=float32)

In [11]:
list(map(lambda x:x.split('/')[-1],test_generator.filenames))

['100017842.jpg',
 '100057293.jpg',
 '1000599894.jpg',
 '1000600110.jpg',
 '1000974891.jpg',
 '1001038363.jpg',
 '1001445041.jpg',
 '1001679799.jpg',
 '100169257.jpg',
 '1001851110.jpg',
 '1004326025.jpg',
 '1005046809.jpg',
 '1005053527.jpg',
 '1005393572.jpg',
 '100586098.jpg',
 '100589879.jpg',
 '1006700844.jpg',
 '100764988.jpg',
 '1008195763.jpg',
 '1008680896.jpg',
 '1009048204.jpg',
 '1009899643.jpg',
 '1010499194.jpg',
 '1011068897.jpg',
 '101108546.jpg',
 '1011126301.jpg',
 '101114208.jpg',
 '101150448.jpg',
 '101166462.jpg',
 '101166704.jpg',
 '101188799.jpg',
 '101375297.jpg',
 '101375898.jpg',
 '1013837646.jpg',
 '1015055767.jpg',
 '101555307.jpg',
 '1015921918.jpg',
 '10161846.jpg',
 '101810058.jpg',
 '1018895399.jpg',
 '1020097694.jpg',
 '1020686893.jpg',
 '1020736703.jpg',
 '1020984282.jpg',
 '1021245884.jpg',
 '1021474962.jpg',
 '1021507836.jpg',
 '1021540518.jpg',
 '1021575808.jpg',
 '102244559.jpg',
 '102568811.jpg',
 '1026095959.jpg',
 '1026346721.jpg',
 '102642691.j

In [28]:
pd.DataFrame(np.argmax(pred, axis=-1)).value_counts()

1    30755
2    24476
0    17547
dtype: int64

In [39]:
import pandas as pd
result = pd.DataFrame(pred)
result.columns = [1, 2, 0]
result.to_csv("/home/kesci/work/incep_819.csv", index=False)
result

,1,2,0
0,0.526464,0.077025,0.396511
1,0.008898,0.040022,0.951080
2,0.174207,0.084389,0.741404
3,0.231672,0.071458,0.696870
4,0.697254,0.302373,0.000373
...,...,...,...
72773,0.000255,0.999638,0.000107
72774,0.377342,0.576091,0.046567
72775,0.371540,0.370029,0.258431
72776,0.658033,0.318002,0.023965


In [37]:
submit

,id,weather
0,100017842.jpg,1
1,100057293.jpg,0
2,1000599894.jpg,0
3,1000600110.jpg,0
4,1000974891.jpg,1
...,...,...
72773,99997209972.jpg,2
72774,99997809978.jpg,2
72775,9999800998.jpg,1
72776,99998609986.jpg,1


In [19]:
submit=pd.DataFrame()
submit['id']=list(map(lambda x:x.split('/')[-1],test_generator.filenames))
submit['weather']=np.argmax(pred, axis=-1)
submit.loc[submit.weather==2,'weather']='sunny'
submit.loc[submit.weather==1,'weather']='others'
submit.loc[submit.weather==0,'weather']='cloudy'

submit.loc[submit.weather=='sunny','weather']=0
submit.loc[submit.weather=='cloudy','weather']=1
submit.loc[submit.weather=='others','weather']=2

In [22]:
submit.weather.value_counts()

2    30755
0    24476
1    17547
Name: weather, dtype: int64

In [23]:
submit.to_csv('/home/kesci/work/incep_submit.csv',index=False)

In [24]:
pd.read_csv('/home/kesci/work/incep_submit.csv')

,id,weather
0,100017842.jpg,1
1,100057293.jpg,0
2,1000599894.jpg,0
3,1000600110.jpg,0
4,1000974891.jpg,1
...,...,...
72773,99997209972.jpg,2
72774,99997809978.jpg,2
72775,9999800998.jpg,1
72776,99998609986.jpg,1


In [101]:
# model=CNN_model()
model=Model()
EarlyStop=EarlyStopping(monitor='val_accuracy',
                        patience=8,verbose=1, mode='auto')
#减小学习率
Reduce=ReduceLROnPlateau(monitor='val_accuracy',
                         factor=0.2,
                         patience=8,
                         verbose=1,
                         mode='auto',
                        #  min_delta=0.0001,
                         cooldown=0,
                         min_lr=0
                         )
checkpoint = ModelCheckpoint(f'/home/kesci/work/cnn_model/mobeilnet_v2.h5',
                            monitor='val_accuracy',
                            verbose=0,
                            mode='auto',
                            save_best_only=True
)


model.fit(x_tra, y_tra,
        validation_data=(x_val,y_val),
        validation_freq=1,
        callbacks=[EarlyStop,Reduce,checkpoint],
        batch_size=64,
        epochs=30)

94773248/94765736 [==============================] - 38s 0us/step
Train on 140466 samples, validate on 35117 samples
Epoch 1/30
140466/140466 [==============================] - 531s 4ms/sample - loss: 0.9414 - accuracy: 0.5920 - val_loss: 0.9717 - val_accuracy: 0.5640
Epoch 2/30
140466/140466 [==============================] - 222s 2ms/sample - loss: 0.9336 - accuracy: 0.6028 - val_loss: 0.9134 - val_accuracy: 0.6284
Epoch 3/30
140466/140466 [==============================] - 219s 2ms/sample - loss: 0.9147 - accuracy: 0.6245 - val_loss: 1.0589 - val_accuracy: 0.4534
Epoch 4/30
140466/140466 [==============================] - 223s 2ms/sample - loss: 0.9090 - accuracy: 0.6331 - val_loss: 0.9100 - val_accuracy: 0.6297
Epoch 5/30
140466/140466 [==============================] - 220s 2ms/sample - loss: 0.9071 - accuracy: 0.6339 - val_loss: 0.9168 - val_accuracy: 0.6203
Epoch 6/30
140466/140466 [==============================] - 223s 2ms/sample - loss: 0.9054 - accuracy: 0.6353 - val_loss: 0

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score
score=f1_score(y_val_df.values, val_pred_df['label1'].values, average='macro')
print("F1_Score:",score)

In [74]:
 index=label[label['label2']<2].index
 x_tra2, x_val2, y_tra2, y_val2 = train_test_split(train[index,],label.loc[label['label2']<2,'label2'], test_size=0.2,shuffle=True,random_state=3090)

In [75]:
model1=CNN_model()
model1.fit(x_tra2, y_tra2,
        validation_data=(x_val2,y_val2),
        validation_freq=1,
        callbacks=[EarlyStop,Reduce,checkpoint],
        batch_size=128,
        epochs=30)

Train on 80656 samples, validate on 20164 samples
Epoch 1/30
80656/80656 [==============================] - 13s 157us/sample - loss: 0.5594 - accuracy: 0.7375 - val_loss: 0.5474 - val_accuracy: 0.7580
Epoch 2/30
80656/80656 [==============================] - 12s 151us/sample - loss: 0.5412 - accuracy: 0.7570 - val_loss: 0.5244 - val_accuracy: 0.7754
Epoch 3/30
80656/80656 [==============================] - 12s 151us/sample - loss: 0.5370 - accuracy: 0.7617 - val_loss: 0.5228 - val_accuracy: 0.7771
Epoch 4/30
80656/80656 [==============================] - 12s 151us/sample - loss: 0.5301 - accuracy: 0.7692 - val_loss: 0.5152 - val_accuracy: 0.7817
Epoch 5/30
80656/80656 [==============================] - 12s 151us/sample - loss: 0.5269 - accuracy: 0.7713 - val_loss: 0.5251 - val_accuracy: 0.7714
Epoch 6/30
80656/80656 [==============================] - 12s 151us/sample - loss: 0.5233 - accuracy: 0.7768 - val_loss: 0.5114 - val_accuracy: 0.7873
Epoch 7/30
80656/80656 [====================

In [84]:
val_pred=np.argmax(model.predict(x_val), axis=-1)
val_pred

array([0, 1, 0, ..., 0, 1, 0])

In [85]:
val_pred_df=pd.DataFrame(val_pred,columns=['label'])
val_pred_df['label1']=2
index1=val_pred_df[val_pred_df['label']==0].index
val_pred_df.loc[index1,'label1']=np.argmax(model1.predict(x_val[index1,]), axis=-1)
val_pred_df

,label,label1
0,0,0
1,1,2
2,0,0
3,0,1
4,0,1
...,...,...
35112,0,0
35113,0,0
35114,0,0
35115,1,2


In [86]:
y_val

33260     0
1687      1
98804     0
41140     0
122824    0
         ..
61203     1
14626     1
35635     1
94678     1
29471     0
Name: label1, Length: 35117, dtype: int64

In [87]:
y_tra_df, y_val_df = train_test_split(label['label2'], test_size=0.2,shuffle=True,random_state=3090)
y_val_df

33260     0
1687      2
98804     0
41140     1
122824    1
         ..
61203     2
14626     2
35635     2
94678     2
29471     1
Name: label2, Length: 35117, dtype: int64

In [88]:
from sklearn.metrics import f1_score
score=f1_score(y_val_df.values, val_pred_df['label1'].values, average='macro')
print("F1_Score:",score)

F1_Score: 0.6584370393697


In [12]:
from sklearn.metrics import f1_score
val_pred=model.predict_classes(x_val)
score=f1_score(y_val, val_pred, average='macro')
print("F1_Score:",score)

# test_loss, test_acc = model.evaluate(x_val,  y_val, verbose=2)
# test_loss,test_acc

F1_Score: 0.66626642137718


In [12]:
pred=model.predict_classes(test)
pred

array([2, 2, 2, ..., 1, 1, 1])

In [36]:
le.inverse_transform(pred)

array(['sunny', 'sunny', 'sunny', ..., 'others', 'others', 'others'],
      dtype=object)

In [17]:
# !wget -nv -O kesci_submit https://cdn.kesci.com/submit_tool/v4/kesci_submit&&chmod +x kesci_submit

2020-10-25 13:35:23 URL:https://cdn.kesci.com/submit_tool/v4/kesci_submit [7357446/7357446] -> "kesci_submit" [1]


In [25]:
# !./kesci_submit -token 7a503d402d41caa3 -file /home/kesci/work/incep_submit.csv

Kesci Submit Tool 4.0.0

> 已验证Token
> 提交文件 /home/kesci/work/incep_submit.csv (1204.74 KiB), Target Qiniu
> 已上传 100 %
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成
